# Data cleaning for olist_sellers_datasets

The dataset was previously extracted from Kaggle and load in Azure Datalake. This python will is to extract the csv files from the Azure Datalake, cleaning the data and load the clean data to Azure Datalake under "transform-data"

In [1]:
# Install package that will be need to provide necessary tools to interact with Azure Data Lake Storage Gen2
!pip install azure-storage-file-datalake
!pip install pyarrow

In [2]:
import os
from pathlib import Path
from azure.storage.filedatalake import DataLakeServiceClient
import pandas as pd
from io import StringIO
from io import BytesIO
import IPython
from IPython.display import display, HTML
import re
import pyarrow as pa
import pyarrow.parquet as pq

In [3]:
#Setup the azure connection 
connection_string = # <-- input the connection here
container_name = "raw-data"
file_path = "sellers.csv"

#Authenticate the connection
service_client = DataLakeServiceClient.from_connection_string(connection_string)

#Get file system and file client
file_system_client = service_client.get_file_system_client(file_system=container_name)
file_client = file_system_client.get_file_client(file_path)

#Download file contents from the raw-data container
download = file_client.download_file()
downloaded_bytes = download.readall()

# Convert to pandas DataFrame
csv_data = downloaded_bytes.decode("utf-8")
sellers= pd.read_csv(StringIO(csv_data))

# Print the df that contains sellers.csv data
print(sellers.head(10000))


                             seller_id  seller_zip_code_prefix  \
0     3442f8959a84dea7ee197c632cb2df15                   13023   
1     d1b65fc7debc3361ea86b5f14c68d2e2                   13844   
2     ce3ad9de960102d0677a81f5d0bb7b2d                   20031   
3     c0f3eea2e14555b6faeea3dd58c1b1c3                    4195   
4     51a04a8a6bdcb23deccc82b0b80742cf                   12914   
...                                ...                     ...   
3090  98dddbc4601dd4443ca174359b237166                   87111   
3091  f8201cab383e484733266d1906e2fdfa                   88137   
3092  74871d19219c7d518d0090283e03c137                    4650   
3093  e603cf3fec55f8697c9059638d6c8eb5                   96080   
3094  9e25199f6ef7e7c347120ff175652c3b                   12051   

            seller_city seller_state  
0              campinas           SP  
1            mogi guacu           SP  
2        rio de janeiro           RJ  
3             sao paulo           SP  
4     bragan

In [4]:
# show the schema of the files
sellers.dtypes

seller_id                 object
seller_zip_code_prefix     int64
seller_city               object
seller_state              object
dtype: object

In [5]:
display(HTML(sellers.head(100).to_html(index=False)))

seller_id,seller_zip_code_prefix,seller_city,seller_state
3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP
c240c4061717ac1806ae6ee72be3533b,20920,rio de janeiro,RJ
e49c26c3edfa46d227d5121a6b6e4d37,55325,brejao,PE
1b938a7ec6ac5061a66a3766e0e75f90,16304,penapolis,SP
768a86e36ad6aae3d03ee3c6433d61df,1529,sao paulo,SP
ccc4bbb5f32a6ab2b7066a4130f114e3,80310,curitiba,PR


### Casting Column to String in Pandas
To cast a column to string type in pandas (when it's being automatically read as an integer), and to add leading zeros to maintain the correct format (like zip codes),

In [6]:
sellers["seller_zip_code_prefix"] = sellers["seller_zip_code_prefix"].astype(str).str.zfill(5)
display(HTML(sellers.head(10).to_html(index=False)))

seller_id,seller_zip_code_prefix,seller_city,seller_state
3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
c0f3eea2e14555b6faeea3dd58c1b1c3,04195,sao paulo,SP
51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP
c240c4061717ac1806ae6ee72be3533b,20920,rio de janeiro,RJ
e49c26c3edfa46d227d5121a6b6e4d37,55325,brejao,PE
1b938a7ec6ac5061a66a3766e0e75f90,16304,penapolis,SP
768a86e36ad6aae3d03ee3c6433d61df,01529,sao paulo,SP
ccc4bbb5f32a6ab2b7066a4130f114e3,80310,curitiba,PR


## Data loading and initial analysis

The scripts starts by loading 'sellers.csv" data from a CSV file into a pandas DataFrame. It then performs an exploratory analysis, including
* Total numbers of records (row)
* Numbers of columns
* Unique cities and states in the dataset
* Missing values

In [7]:
# Print dataset shape
print(f"\nDataset contains {sellers.shape[0]:,} records and {sellers.shape[1]} columns.")

# Count unique seller cities and states
unique_cities = sellers['seller_city'].nunique()
unique_states = sellers['seller_state'].nunique()
print(f"Number of unique cities: {unique_cities}")
print(f"Number of unique states: {unique_states}")




Dataset contains 3,095 records and 4 columns.
Number of unique cities: 611
Number of unique states: 23


### Check missing values and duplicate records

In [8]:
# Count missing values in each column
missing_values = sellers.isnull().sum()

# Calculate percentage of missing values
missing_pct = (missing_values / len(sellers)) * 100

# Combine into a DataFrame 
missing_report = pd.DataFrame({
    'Missing Values': missing_values,
    'Percentage (%)': missing_pct
    })

# Check for duplicate records on seller_id values
duplicate_seller_id_count = sellers.duplicated(subset='seller_id', keep=False).sum()

# Show only columns with missing values
print("\nMissing Values Analysis:")
print(missing_report[missing_report['Missing Values'] > 0])
print(f"Number of duplicate seller_id records: {duplicate_seller_id_count}")


Missing Values Analysis:
Empty DataFrame
Columns: [Missing Values, Percentage (%)]
Index: []
Number of duplicate seller_id records: 0


## Data Cleaning Process
The dataset undergoes detail cleaning through following steps

#### a) Basic String Cleaning:
* Trims leading/trailing whitespace from all string columns

#### b) Enhance City Name Cleaning:
* Removes numeric values and special charactes
* Standardizes city name by
    * Splitting entries on common delimiters
    * Converting special characters (e.g., replacing "são" with "sao"

### c) State Name Standardise
* Concerts all state names to uppercase format


In [9]:
# a) Basic string cleaning: trim whitespace
sellers['seller_city'] = sellers['seller_city'].astype(str).str.strip()
sellers['seller_state'] = sellers['seller_state'].astype(str).str.strip()

# b) Define Portuguese-aware title case
def custom_title(text):
    exceptions = {'de', 'da', 'do', 'das', 'dos', 'e'}
    special_cases = {"d'oeste": "d'Oeste", "d'alianca": "d'Alianca"}
    words = text.lower().split()
    result_words = []
    for i, word in enumerate(words):
        if word in special_cases:
            result_words.append(special_cases[word])
            continue
        if '-' in word:
            subwords = word.split('-')
            new_subwords = [
                sub.capitalize() if (j == 0 or sub not in exceptions) else sub
                for j, sub in enumerate(subwords)
            ]
            result_words.append('-'.join(new_subwords))
        else:
            result_words.append(word.capitalize() if i == 0 or word not in exceptions else word)
    return ' '.join(result_words)

# c) Clean and normalize seller_city
def clean_city_name(city):
    if pd.isnull(city):
        return city
    city = city.lower().strip()
    city = re.sub(r'\d+', '', city)                     # Remove digits
    city = re.sub(r'[^\w\s\-]', '', city)               # Remove special characters except hyphen
    city = re.split(r'[,;]', city)[0].strip()           # Keep first part before comma/semicolon
    city = city.replace('ã', 'a').replace('â', 'a')
    city = city.replace('á', 'a').replace('à', 'a')
    city = city.replace('é', 'e').replace('ê', 'e')
    city = city.replace('í', 'i')
    city = city.replace('ó', 'o').replace('ô', 'o')
    city = city.replace('ú', 'u')
    city = city.replace('ç', 'c')
    return custom_title(city)

# Apply cleaning to seller_city
sellers['seller_city'] = sellers['seller_city'].apply(clean_city_name)

# d) Standardise state name to uppercase
sellers['seller_state'] = sellers['seller_state'].str.upper()

# Display first 10 cleaned rows
display(HTML(sellers.head(10).to_html(index=False)))

seller_id,seller_zip_code_prefix,seller_city,seller_state
3442f8959a84dea7ee197c632cb2df15,13023,Campinas,SP
d1b65fc7debc3361ea86b5f14c68d2e2,13844,Mogi Guacu,SP
ce3ad9de960102d0677a81f5d0bb7b2d,20031,Rio de Janeiro,RJ
c0f3eea2e14555b6faeea3dd58c1b1c3,04195,Sao Paulo,SP
51a04a8a6bdcb23deccc82b0b80742cf,12914,Braganca Paulista,SP
c240c4061717ac1806ae6ee72be3533b,20920,Rio de Janeiro,RJ
e49c26c3edfa46d227d5121a6b6e4d37,55325,Brejao,PE
1b938a7ec6ac5061a66a3766e0e75f90,16304,Penapolis,SP
768a86e36ad6aae3d03ee3c6433d61df,01529,Sao Paulo,SP
ccc4bbb5f32a6ab2b7066a4130f114e3,80310,Curitiba,PR


In [10]:
# Display top 20 cleaned city and state values

#Group by seller_stae and count sellers
state_summary = (
    sellers.groupby('seller_state')
    .size()
    .reset_index(name='sellers')
    .sort_values(by='sellers', ascending=False)
)

# Add market share percentage column
state_summary['market_share_pct'] = (state_summary['sellers'] / state_summary['sellers'].sum() * 100).round(2)

# display the result (only top 20 rows)
display(HTML(state_summary.head(20).to_html(index=False)))

seller_state,sellers,market_share_pct
SP,1849,59.74
PR,349,11.28
MG,244,7.88
SC,190,6.14
RJ,171,5.53
RS,129,4.17
GO,40,1.29
DF,30,0.97
ES,23,0.74
BA,19,0.61


#### Check total in sellers_clean DataFrame before upload

In [11]:
print(f"Total row in sellers_clean DataFrame: {len(sellers):,}")

Total row in sellers_clean DataFrame: 3,095


### Save cleaned dataset to local as csv file

In [12]:
sellers.to_csv("cleaned_sellers.csv", index=False)
print("The CSV files has saved locally as 'clean_sellers.csv'")

The CSV files has saved locally as 'clean_sellers.csv'


### Save cleaned dataset to Azure Data Lake (transform-data) as parquet ffile

In [13]:
# Convert to in-memory parquet
buffer = BytesIO()
pq.write_table(pa.Table.from_pandas(sellers), buffer)
buffer.seek(0)

# Azure connection config
connection_string = "DefaultEndpointsProtocol=https;AccountName=jde06dark;AccountKey=5dPJ4+PvxUP5h+GnoaiXU0+I288QJ8qFQSlMV6V8FJvXdgR02ut4Acg/8vC+7FrJQtrsP66a7W6x+AStovZzaw==;EndpointSuffix=core.windows.net"
container_name_clean = "transform-data"
file_path_clean = "cleaned_sellers.parquet"

# connect to ADLS
service_client = DataLakeServiceClient.from_connection_string(connection_string)
fs_client = service_client.get_file_system_client(container_name_clean)

# Delete existing file if exists
try:
    fs_client.delete_file(file_path_clean)
except:
    pass  # ignore if file not found

# Upload new parquet file
file_client = fs_client.create_file(file_path_clean)
file_client.append_data(buffer.read(), offset=0, length=buffer.getbuffer().nbytes)
file_client.flush_data(buffer.getbuffer().nbytes)

print(f" Parquet file uploaded to ADLS under '{container_name_clean}/{file_path_clean}'")

 Parquet file uploaded to ADLS under 'transform-data/cleaned_sellers.parquet'


#### Verify row count after upload 

In [14]:
# Download file from Azure
file_client = fs_client.get_file_client(file_path_clean)
download = file_client.download_file()
data = download.readall()

# Load into DataFrame
buffer = BytesIO(data)
table = pq.read_table(buffer)
df_parquet = table.to_pandas()

# Print the row count
print(f" Rows in uploaded Parquet file: {len(df_parquet):,}")

 Rows in uploaded Parquet file: 3,095
